In [ ]:
import os
import numpy as np
import sys
# Drawing plots is not supported in sensi.yml environmental
# import matplotlib
# import matplotlib.pyplot as plt
# import statsmodels
# import cartopy.crs as ccrs
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
import shap
from pathos.multiprocessing import ProcessingPool as Pool
from sklearn.linear_model import LinearRegression, TheilSenRegressor
import regressors
import regressors.stats as regressors_stats
import time
import warnings
warnings.filterwarnings('ignore')

# limit number of threads
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREAD"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

# plt.rcParams.update({'font.size': 6})

In [ ]:
lat = 360
lon = 720
year1 = 0  # 1982
year2 = 36  # 2017
year = 36
mon = 12
features = ['tp', 'sm1', 'sm2', 't2m', 'ssrd', 'vpd']

def data_path(filename):
    file_path = "{path}/{filename}".format(
        path="/Net/Groups/BGI/scratch/wantong/study2/data_to_upload",
        filename=filename
    )
    return file_path

def read_data(path):
    data = np.load(path)
    print(path, 'read data')
    return data

def graph(ax, target, **kwargs):
    lon = np.arange(-180,180,0.5)
    lat = np.arange(-90,90,0.5)
    ax.coastlines(linewidth=0.2)
    ax.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())
    cs = ax.pcolor(lon, lat, target[::-1,:], transform=ccrs.PlateCarree(), **kwargs)
    cbar = ax.figure.colorbar(cs, ax=ax, shrink=0.4, extend='both')
    return(cs)

def minus_month_mean(lai, tp, s1, s2, t2m, ssrd, vpd):
    mean = np.zeros((mon, 7, lat, lon))
    ndvi_without = np.zeros((year*mon, 7, lat, lon))
    for row in range(lat):
        for col in range(lon):
            if np.isnan(lai[:, row, col]).all():
                ndvi_without[:, :, row, col] = np.nan
            else:
                for m in range(mon):
                    sum1 = 0
                    sum2 = 0
                    sum3 = 0
                    sum4 = 0
                    sum5 = 0
                    sum6 = 0
                    sum0 = 0
                    i = 0
                    for y in range(year):
                        if ~np.isnan(lai[m + mon * y, row, col]):
                            sum0 = sum0 + lai[m + mon * y, row, col]
                            sum1 = sum1 + tp[m + mon * y, row, col]
                            sum2 = sum2 + s1[m + mon * y, row, col]
                            sum3 = sum3 + s2[m + mon * y, row, col]
                            sum4 = sum4 + t2m[m + mon * y, row, col]
                            sum5 = sum5 + ssrd[m + mon * y, row, col]
                            sum6 = sum6 + vpd[m + mon * y, row, col]
                            i = i + 1
                    if i == 0:
                        mean[m, :, row, col] = np.nan
                    else:
                        mean[m, 0, row, col] = sum0/i
                        mean[m, 1, row, col] = sum1/i
                        mean[m, 2, row, col] = sum2/i
                        mean[m, 3, row, col] = sum3/i
                        mean[m, 4, row, col] = sum4/i
                        mean[m, 5, row, col] = sum5/i
                        mean[m, 6, row, col] = sum6/i

    for y in range(year):
        for m in range(mon):
            ndvi_without[mon * y + m, 0, :, :] = lai[mon * y + m, :, :] - mean[m, 0, :, :]
            ndvi_without[mon * y + m, 1, :, :] = tp[mon * y + m, :, :] - mean[m, 1, :, :]
            ndvi_without[mon * y + m, 2, :, :] = s1[mon * y + m, :, :] - mean[m, 2, :, :]
            ndvi_without[mon * y + m, 3, :, :] = s2[mon * y + m, :, :] - mean[m, 3, :, :]
            ndvi_without[mon * y + m, 4, :, :] = t2m[mon * y + m, :, :] - mean[m, 4, :, :]
            ndvi_without[mon * y + m, 5, :, :] = ssrd[mon * y + m, :, :] - mean[m, 5, :, :]
            ndvi_without[mon * y + m, 6, :, :] = vpd[mon * y + m, :, :] - mean[m, 6, :, :]
    print('Without_seasonality done')
    return(ndvi_without)

def grow_season(threshold, lai_with, t2m, without_seasonality):
    all_white = np.zeros((year * mon, 7, lat, lon), dtype=np.float32)
    kk = np.zeros((year + 1, lat, lon), dtype=np.int)
    for row in range(lat):
        for col in range(lon):
            if np.isnan(without_seasonality[:, 0, row, col]).all():
                all_white[:, :, row, col] = np.nan
            else:
                i = 0
                for y in range(year):
                    for m in range(mon):
                        if lai_with[y * mon + m, row, col] > threshold \
                                and t2m[y * mon + m, row, col] + 273.15 > 5 \
                                and ~np.isnan(without_seasonality[y * mon + m, :, row, col]).any() \
                                and ~np.isinf(without_seasonality[y * mon + m, :, row, col]).any():
                            all_white[i, :, row, col] = without_seasonality[y * mon + m, :, row, col]
                            i = i + 1
                    kk[y + 1, row, col] = i
                if i == 0:
                    all_white[:, :, row, col] = np.nan
    print('Grow_season done')
    return (all_white, kk)

def lowess_func(grow,kk):
    lowess_cal = sm.nonparametric.lowess
    yearlist = np.zeros((year * mon), dtype=np.float32)
    for i in range(year * mon):
        yearlist[i] = i
    grow_lowess = np.zeros((year * mon, 2, 7, lat, lon), dtype=np.float32)
    sub_lowess = np.zeros((year * mon, 7, lat, lon), dtype=np.float32)
    for row in range(360):
        print('row:',row)
        for col in range(720):
            k=kk[year,row,col]
            if k < 5 or np.isnan(grow[0:k,:,row,col]).any():
                sub_lowess[:, :, row, col] = np.nan
            else:
                for t in range(7):
                    grow_lowess[0:k, :, t, row, col] = lowess_cal(grow[0:k, t, row, col], yearlist[0:k], frac=0.4)
                    sub_lowess[0:k, t, row, col] = grow[0:k, t, row, col] - grow_lowess[0:k, 1, t, row, col]
    print('detrend done')
    return(sub_lowess)

def RF_longterm(lowe,kk,lowe1,kk1,lowe2,kk2):
    output = np.zeros((2, 6, lon), dtype=np.float32) * np.nan # first dimension: results of slope and p_value; second dimension: 6 hydro-climate variables; third dimension: longtitude
    
    # iterate over data across longtitudes
    lon_new = [340,400] # European domain
    # lon_new = [0,720] # Global domain
    for col in range(lon_new[0] + 1, lon_new[1] - 1, 1):
        k0 = kk[1, col] # data index for the first year
        k5 = kk[year, col] # data index for the final year
        if np.isnan(lowe[k0:k5, :, col]).any() or np.isinf(lowe[k0:k5, :, col]).any() or np.all(
                lowe[k0:k5, :, col] == 0):
            output[:, :, col] = np.nan
        else:
            # 3x3 grid cells need to be used in one random forest model
            test_data1 = lowe1[kk1[1, col - 1]:kk1[year, col - 1], :, col - 1]
            test_data2 = lowe[kk[1, col - 1]:kk[year, col - 1], :, col - 1]
            test_data3 = lowe2[kk2[1, col - 1]:kk2[year, col - 1], :, col - 1]
            test_data4 = lowe1[kk1[1, col]:kk1[year, col], :, col]
            test_data5 = lowe[kk[1, col]:kk[year, col], :, col]
            test_data6 = lowe2[kk2[1, col]:kk2[year, col], :, col]
            test_data7 = lowe1[kk1[1, col + 1]:kk1[year, col + 1], :, col + 1]
            test_data8 = lowe[kk[1, col + 1]:kk[year, col + 1], :, col + 1]
            test_data9 = lowe2[kk2[1, col + 1]:kk2[year, col + 1], :, col + 1]
            test_data11 = np.concatenate((test_data1, test_data2, test_data3, test_data4,
                                          test_data5, test_data6, test_data7, test_data8, test_data9))
            test_data = test_data11[~np.all(test_data11 == 0, axis=1)]
            test_data = test_data[~np.any(np.isnan(test_data), axis=1)]

            if len(test_data[:, 0]) >= 50: # maximum data points can be 9 grid cells * 12 months * 36 years without considering growing seasons and data gaps
                # set common hyperparameters to train a random forest model
                rf = RandomForestRegressor(n_estimators=100,
                                           max_features=0.3,
                                           n_jobs=1,
                                           bootstrap=True,
                                           oob_score=True,
                                           random_state=42)

                rf.fit(test_data[:, 1:7], test_data[:, 0])
                
                if rf.oob_score_ > 0: # set a basic threshold of model performance
                    explainer = shap.TreeExplainer(rf)
                    shap_values = explainer.shap_values(test_data[:, 1:7])
                    
                    for v in [2]: # only calculate SHAP values of LAI to sub-surface soil moisture
                        y = shap_values[:, v]
                        x = test_data[:, v + 1]
                        x = x.reshape(-1, 1)
                        x_new = x[~np.isnan(y)]
                        y_new = y[~np.isnan(y)]
                        if len(y_new) >= 5 and ~np.isnan(x_new).any() and ~np.isinf(x_new).any() and ~np.all(x_new == 0):
                            tel = TheilSenRegressor().fit(x_new, y_new)
                            output[0, v, col] = tel.coef_ #Theil-sen slope
                            output[1, v, col] = regressors_stats.coef_pval(tel, x_new, y_new)[1] # p_value
                            print('oob:', rf.oob_score_, 'sensitivity:', output[0, v, col],'p_value:',output[1, v, col])

    return (output)

def RF_3yblock(lowe,kk,lowe1,kk1,lowe2,kk2):
    output = np.zeros((year, 6, lon), dtype=np.float32) * np.nan
    
    # iterate over data across longtitudes
    lon_new = [340,400] # European domain
    # lon_new = [0,720] # Global domain
    for col in range(lon_new[0] + 1, lon_new[1] - 1, 1):
        block = 3
        for move in range(year1, year2, block):
            # 3x3 grid cells need to be used in one random forest model
            test_data1 = lowe1[kk1[move, col - 1]:kk1[move+block, col - 1], :, col-1]
            test_data2 = lowe[kk[move, col - 1]:kk[move+block, col - 1], :, col-1]
            test_data3 = lowe2[kk2[move, col - 1]:kk2[move+block, col - 1], :, col-1]
            test_data4 = lowe1[kk1[move, col]:kk1[move+block, col], :, col]
            test_data5 = lowe[kk[move, col]:kk[move+block, col], :, col]
            test_data6 = lowe2[kk2[move, col]:kk2[move+block, col], :, col]
            test_data7 = lowe1[kk1[move, col + 1]:kk1[move+block, col + 1], :, col+1]
            test_data8 = lowe[kk[move, col + 1]:kk[move+block, col + 1], :, col+1]
            test_data9 = lowe2[kk2[move, col + 1]:kk2[move+block, col + 1], :, col+1]
            test_data11 = np.concatenate((test_data1,test_data2,test_data3,test_data4,
                                            test_data5,test_data6,test_data7,test_data8,test_data9))
            test_data = test_data11[~np.all(test_data11 == 0, axis=1)]
            test_data = test_data[~np.any(np.isnan(test_data), axis=1)]

            if np.isnan(test_data).any() or np.isinf(test_data).any() or np.all(test_data == 0):
                output[:, :, col] = np.nan
            else:
                if len(test_data[:, 0]) >= 15: # maximum data points can be 9 grid cells * 12 months * 3 years without considering growing seasons and data gaps
                    # set common hyperparameters to train a random forest model
                    rf = RandomForestRegressor(n_estimators=100,
                                           max_features=0.3,
                                           n_jobs=1,
                                           bootstrap=True,
                                           oob_score=True,
                                           random_state=42)

                    rf.fit(test_data[:, 1:7], test_data[:, 0])

                    if rf.oob_score_ > 0: # set a basic threshold of model performance
                        explainer = shap.TreeExplainer(rf)
                        shap_values = explainer.shap_values(test_data[:, 1:7])
                    
                        for v in [2]: # only calculate SHAP values of LAI to sub-surface soil moisture
                            y=shap_values[:, v]
                            x=test_data[:, v+1]
                            x = x.reshape(-1, 1)
                            x_new = x[~np.isnan(y)]
                            y_new = y[~np.isnan(y)]
                            if len(y_new) >= 5 and ~np.isnan(x_new).any() and ~np.isinf(x_new).any() and ~np.all(x_new == 0):
                                tel = TheilSenRegressor().fit(x_new, y_new)
                                output[move, v, col] = np.round(tel.coef_,7)
                                pvalue = np.round(regressors_stats.coef_pval(tel, x_new, y_new)[1],7)
                                if pvalue >= 0.1 or pvalue < 0: # significance test
                                    output[move, v, col] = np.nan
                            print('oob:', rf.oob_score_, 'sensitivity:', output[move, v, col])

    return (output)


In [ ]:
# load LAI from 1982 to 2017
LAI_globe = read_data(data_path('test/LAI3g_1982_2018_monthly_0.5.npy'))[0:432,:,:]

# To shorten the calculating time, we can use European domain as demo, and set data in the other regions as NaN
LAI = np.zeros((432,360,720)) * np.nan
LAI[:,40:100,340:400] = LAI_globe[:,40:100,340:400]

# Drawing plots is not supported in sensi.yml environmental
# show demo domain
# fig = plt.figure(figsize=[2,2], dpi=300, tight_layout=True)
# ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
# graph(ax, np.nanmean(LAI,axis=0), norm=matplotlib.colors.Normalize(vmin=0, vmax=10), cmap='Reds')
# plt.show()


In [ ]:
# load hydro-climate variables (near-surface soil moisture, sub-surface soil moisture, precipitation, temperature, vapor pressure deficit, solar radiation)
SM1 = read_data('/Net/Groups/BGI/scratch/wantong/study2/original_data/ERA5-Land/0d50_monthly/ERA5-land-mm_SMsurf_1982_2017_monthly_0d50.npy')
SM2 = read_data('/Net/Groups/BGI/scratch/wantong/study2/original_data/ERA5-Land/0d50_monthly/ERA5-land-mm_SMroot_1982_2017_monthly_0d50.npy')
TP = read_data('/Net/Groups/BGI/scratch/wantong/study2/original_data/TRENDY_climate/CRUJRApre_444_0.5.npy')
T2M = read_data('/Net/Groups/BGI/scratch/wantong/study2/original_data/TRENDY_climate/CRUJRAtmp_444_0.5.npy')
SSRD = read_data('/Net/Groups/BGI/scratch/wantong/study2/original_data/TRENDY_climate/CRUJRAdswrf_444_0.5.npy') / (1000000 / 86000) # convert unit from W/m2 to MJ/m2
VPD = read_data('/Net/Groups/BGI/scratch/wantong/study2/original_data/TRENDY_climate/CRUJRAvpd_444_0.5.npy')

LAI[LAI < 0] = np.nan
SM1[SM1 < 0] = np.nan # mm
SM1[SM1 > 10000] = np.nan
SM2[SM2 < 0] = np.nan # mm
SM2[SM2 > 10000] = np.nan
TP[TP < 0] = np.nan # mm
TP[TP > 100000] = np.nan
T2M[T2M < 0] = np.nan  # Kelvin scale
T2M[T2M > 500] = np.nan
SSRD[SSRD < 0] = np.nan
SSRD[SSRD > 10000] = np.nan
VPD[VPD < 0] = np.nan
VPD[VPD > 10] = np.nan  # KPa


In [ ]:
# Remove mean seasonal cycles
Without_seasonality = minus_month_mean(LAI[0:432, :,:], TP[0:432, :,:], SM1[0:432, :,:], SM2[0:432, :,:],
                                       T2M[0:432, :,:],SSRD[0:432, :,:], VPD[0:432, :,:])

# Return data and index for each year in the growing seasons (LAI>0.5 and T2M>5˚C)
Grow_season, KK = grow_season(0.5, LAI[0:432, :,:], T2M[0:432, :,:], Without_seasonality)
np.save(data_path('test/LAIano_6v_monthly_1982_2017_0d50_Grow_season'), Grow_season)
np.save(data_path('test/LAIano_6v_monthly_1982_2017_0d50_KK'), KK)

In [ ]:
# ~ 15 mins runtime
# Remove long-term trends
detrend = lowess_func(Grow_season, KK)
np.save(data_path('test/LAIano_6v_monthly_1982_2017_0d50_lowess0.4'), detrend)

In [ ]:
# ~ 35 mins runtime
start_time = time.time()

# calculate LAI sensitivity to sub-surface soil moisture from 3-year blocks using random forests and SHAP values
detrend = read_data(data_path('test/LAIano_6v_monthly_1982_2017_0d50_lowess0.4.npy'))
KK = read_data(data_path('test/LAIano_6v_monthly_1982_2017_0d50_KK.npy'))

# set CPU usage
pool = Pool(16)

# data-preparation for multiprocessing: iterate over data across latitudes for 3 rows (in the end, 3x3 grid cells need to be used in one random forest model.)
lat_new = [40,100] # European domain
# lat_new = [0,360] # Global domain
Lowe = [detrend[:, :, row,:] for row in range(lat_new[0]+1, lat_new[1]-1, 1)]
KKK = [KK[:,row,:] for row in range(lat_new[0]+1, lat_new[1]-1, 1)]
Lowe1 = [detrend[:, :, row,:] for row in range(lat_new[0]+0, lat_new[1]-2,1)]
KKK1 = [KK[:,row,:] for row in range(lat_new[0]+0, lat_new[1]-2, 1)]
Lowe2 = [detrend[:, :, row,:] for row in range(lat_new[0]+2, lat_new[1], 1)]
KKK2 = [KK[:,row,:] for row in range(lat_new[0]+2, lat_new[1], 1)]

# multiprocessing of function RF_3yblock
outs = pool.map(RF_3yblock, Lowe, KKK, Lowe1, KKK1, Lowe2, KKK2)
pool.close()
pool.join()
print('shape(outs):',np.shape(outs))

SHAP_3Y = np.zeros((year, 6, lat, lon)) * np.nan
# upload results across latitudes
for part in range(1, lat_new[1]-lat_new[0]-1, 1):
    SHAP_3Y[:, :, lat_new[0]+part, :] = outs[part-1]

np.save(data_path('test/LAIano_6v_monthly_1982_2017_0d50_ContributionSensitivity_3Yblock'), SHAP_3Y)

print("16 cpus--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# ~ ??? min runtime
start_time = time.time()

# calculate LAI sensitivity to sub-surface soil moisture from 1982 to 2017 using random forests and SHAP values
detrend = read_data(data_path('test/LAIano_6v_monthly_1982_2017_0d50_lowess0.4.npy'))
KK = read_data(data_path('test/LAIano_6v_monthly_1982_2017_0d50_KK.npy'))

# set CPU usage
pool = Pool(16)

# data-preparation for multiprocessing: iterate over data across latitudes for 3 rows (in the end, 3x3 grid cells need to be used in one random forest model.)
lat_new = [40,100] # European domain
# lat_new = [0,360] # Global domain
Lowe = [detrend[:, :, row,:] for row in range(lat_new[0]+1, lat_new[1]-1, 1)]
KKK = [KK[:,row,:] for row in range(lat_new[0]+1, lat_new[1]-1, 1)]
Lowe1 = [detrend[:, :, row,:] for row in range(lat_new[0]+0, lat_new[1]-2, 1)]
KKK1 = [KK[:,row,:] for row in range(lat_new[0]+0, lat_new[1]-2, 1)]
Lowe2 = [detrend[:, :, row,:] for row in range(lat_new[0]+2, lat_new[1], 1)]
KKK2 = [KK[:,row,:] for row in range(lat_new[0]+2, lat_new[1], 1)]

# multiprocessing of function RF_longterm
outs = pool.map(RF_longterm, Lowe, KKK, Lowe1, KKK1, Lowe2, KKK2)
pool.close()
pool.join()
print('shape(outs):',np.shape(outs))

SHAP_all = np.zeros((2, 6, lat, lon)) * np.nan
# upload results across latitudes
for part in range(1, lat_new[1]-lat_new[0]-1, 1):
    SHAP_3Y[:, :, lat_new[0]+part, :] = outs[part-1]
np.save(data_path('test/LAIano_6v_monthly_1982_2017_0d50_ContributionSensitivity_allYear'), SHAP_all)

print("16 cpus--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Visulization
# Drawing plots is not supported in sensi.yml environmental. Please change your environment
import matplotlib
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

def europe_map(data, ax, min, max):
    ax.coastlines(linewidth=0.1)
    ax.set_extent([-10, 40, 35, 70], crs=ccrs.PlateCarree())
    lon = np.arange(-10,40,0.5)
    lat = np.arange(35,70,0.5)
    if max>0 and min<0:
        norm = matplotlib.colors.TwoSlopeNorm(vmin=min, vcenter=0, vmax=max)
        cmap = 'coolwarm'
    elif max<=0:
        norm = matplotlib.colors.Normalize(vmin=min, vmax=max)
        cmap = truncate_colormap(plt.get_cmap('coolwarm'), 0, 0.5)
    elif min>=0:
        norm = matplotlib.colors.Normalize(vmin=min, vmax=max)
        cmap = truncate_colormap(plt.get_cmap('coolwarm'), 0.5, 1)
    map = ax.pcolormesh(lon, lat, data[::-1,:], cmap = cmap, norm=norm, transform=ccrs.PlateCarree())
    cbar = ax.figure.colorbar(map, ax=ax, ticks=[min,max], shrink=0.4, extend='both')
    
    return(cbar)

data = read_data(data_path('test/LAIano_6v_monthly_1982_2017_0d50_ContributionSensitivity_allYear.npy')
data[0,2,:,:][data[1,2,:,:]>=0.01] = np.nan
fig = plt.figure(figsize=(5, 2), dpi=300, tight_layout=True)
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
europe_map(data[0,2,40:110,340:440], ax, min=-0.005, max=0.005)
ax.set_title('Overall sensitivity of LAI to sub-surface soil moisture from 1982 to 2017')

